In this notebook, I show coupled simulations with a white noise stochastic parameterization. This noise has a covariance structure between QT and SLI given by the error made by the base neural network. This covariance depends on latitude, but the noise is completely white in space and time.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')
from src.data.sam import SAMRun



def report_2d(data_2d):
    """Report of 2D data"""
    
    # make coordinates have units of 1000s of km 
    data_2d = data_2d.assign_coords(y=data_2d.y/1e6, x=data_2d.x/1e6)
    
    
    fig, axs = plt.subplots(3, 2, constrained_layout=True, figsize=(10, 9))
    axs = axs.ravel()
    
    i = 0
    data_2d.PW.mean('x').T.plot(ax=axs[i])
    axs[i].set_title('Zonal Mean PW')
    
    i +=1
    data_2d.PW.std('x').T.plot(ax=axs[i])
    axs[i].set_title('Zonal Std PW')
    
    i +=1
    data_2d.NPNN.mean('x').T.plot(ax=axs[i])
    axs[i].set_title('Net Precip Zonal Mean (from QT)')
    
    i +=1
    data_2d.W500.mean('x').T.plot(ax=axs[i])
    axs[i].set_title('W500 Zonal mean')
    
    i +=1
    t = - 1
    time = float(data_2d.time[t])
    data_2d.PW[t].plot(ax=axs[i])
    axs[i].set_title(f'PW Snapshot at {time}')
    
    i +=1
    data_2d.NPNN[t].plot(ax=axs[i])
    axs[i].set_title(f'Net Precip (QT) Snapshot at {time}')
    
    for ax in axs[i+1:]:
        ax.axis('off')

# Noisy run

In [ ]:
run_noise = SAMRun('../data/runs/2019-03-14-model268-epoch5-debiased-noise/', case='control')

report_2d(run_noise.data_2d)

# Non-noisy run

In [ ]:
run_orig = SAMRun('../data/runs/2019-03-14-model268-epoch5-debiased/', case='control')
report_2d(run_orig.data_2d)

# Comparisons

In [ ]:
from src.utils import get_regions
from src.data import open_data
from uwnet.thermo import lhf_to_evap

training = open_data('ngaqua_2d').sel(time=slice(100, 110))
training['NPNN'] = training.Prec - lhf_to_evap(training.LHF)

In [ ]:
def groupby_region(data_2d):
    regions = get_regions(data_2d.y)
    return data_2d.groupby(regions)

def plot_tropical_avg(a, **kwargs):
    groupby_region(a).mean(['y', 'x']).sel(y='Tropics').plot(**kwargs)

def plot_tropical_std(a, **kwargs):
    groupby_region(a).std(['y', 'x']).sel(y='Tropics').plot(**kwargs)
    
def compare_avg(field):
    plot_tropical_avg(training[field], label='NG-Aqua')
    plot_tropical_avg(run_orig.data_2d[field], label='Orig')
    plot_tropical_avg(run_noise.data_2d[field], label='Noise')
    unit = str(run_orig.data_2d[field].units)

    
    plt.title(f"Average {field} in Tropics ({unit})")
    plt.legend()
    
def compare_std(field):
    plot_tropical_std(training[field], label='NG-Aqua')
    plot_tropical_std(run_orig.data_2d[field], label='Orig')
    plot_tropical_std(run_noise.data_2d[field], label='Noise')
    unit = str(run_orig.data_2d[field].units)
    plt.title(f"StdDev {field} in Tropics ({unit})")
    plt.legend()

In [ ]:
compare_avg('PW')

In [ ]:
compare_std('PW')

In [ ]:
compare_avg('NPNN')

In [ ]:
compare_avg('W500')

Clearly none of the scheme does not produce enough vertical motion in the tropics, perhaps because the average precipitation is too low.